####  <span style="color:#5D8BF4"> Irena Vent </span>

# <span style="color:#051367"> Machine Learning</span>

<span style="color:#051367"> **Dataset**: Airbnb </span>

<span style="color:#051367"> **Objetivo**: Predecir el precio de un airbnb situado Madrid</span>


In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

import warnings
warnings.filterwarnings('ignore')

In [3]:
def plot_confusion_matrix(confmat):
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.5)
    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')

    plt.xlabel('predicted label')
    plt.ylabel('true label')

    plt.tight_layout()
    plt.show()

 <span style="color:#5D8BF4"> Cargamos nuestro dataset con el objetivo de: </span>
    
1. Filtrar el dataset por Madird (Spain); 
2. Comprobar si existen variables que puedan ser descartadas previo al análisis exploratior de datos, por ejemplo: ids, urls, etc.

In [4]:
prueba = pd.read_csv('./data/airbnb-listings.csv', sep=';', decimal='.')

#pd.set_option('display.max_columns', None)
#prueba.head()

In [5]:
prueba.shape

(14780, 89)

In [6]:
# eliminamos las muestras que contienen NaN en las columnas indicadas
prueba = prueba.dropna(subset=['Country', 'City'])

# filtramos por Spian y Madrid
prueba = prueba.loc[prueba['Country'] == 'Spain']
prueba = prueba[prueba["City"].str.contains("Madrid")]

#print(f'Dimensión dataframe --> {prueba.shape}')
#print(f'Valores únicos en Country --> {prueba["Country"].unique()}')
#print(f'Valores únicos en City --> {prueba["City"].unique()}')

# eliminamos columnas
prueba = prueba.drop(['ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary', 'Space', 'Description',
                      'Experiences Offered', 'Neighborhood Overview','Zipcode' , 'Notes', 'Transit', 'Access', 
                      'Interaction', 'House Rules', 'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url',
                      'Host ID', 'Host URL', 'Host Name', 'Host Since', 'Host Location', 'Host About','Amenities', 
                      'Host Thumbnail Url', 'Host Picture Url', 'Host Neighbourhood','Host Listings Count', 
                      'Host Total Listings Count', 'Host Verifications', 'Street', 'City', 'State', 'Market',
                      'Smart Location', 'Country Code', 'Country', 'First Review', 'Last Review', 'Calendar Updated', 
                      'Calendar last Scraped', 'License', 'Jurisdiction Names', 'Calculated host listings count',
                      'Geolocation', 'Features'], axis = 1)

prueba.shape

(13234, 40)

<span style="color:#5D8BF4"> **Divsión del dataset en train y test** para el desarrollo de análisis exploratorio de datos. Que se realizará únicamente sobre el subconjunto de train. Posteriormente, en la validación del modelo elegido, las decisiones tomadas sobre train se aplicaran al subconjunto de test. </span> 

In [7]:
from sklearn.model_selection import train_test_split

#full_df = pd.read_csv('./airbnb-listings-extract.csv', sep=';', decimal='.')
train, test = train_test_split(prueba, test_size=0.2, shuffle=True, random_state=0)

print(f'Dimensiones del dataset de training: {train.shape}')
print(f'Dimensiones del dataset de test: {test.shape}')

# Guardamos
train.to_csv('./train.csv', sep=';', decimal='.', index=False)
test.to_csv('./test.csv', sep=';', decimal='.', index=False)

# A partir de este momento cargamos el dataset de train y trabajamos ÚNICAMENTE con él. 
df_train = pd.read_csv('./train.csv', sep=';', decimal='.')

Dimensiones del dataset de training: (10587, 40)
Dimensiones del dataset de test: (2647, 40)


<span style="color:#5D8BF4"> **Análisis exploratior de los datos** </span> 

In [ ]:
pd.set_option('display.max_columns', None)
prueba.head()

In [ ]:
df_train.describe().T

In [ ]:
#df_train.isnull().any()
df_train.isnull().sum()

In [ ]:
df_train.dtypes

<span style="color:#5D8BF4"> **Primeras observaciones**</span>

- Existen varibles, como son: Host Response Time, Neighbourhood, Neighbourhood Cleansed, Neighbourhood Group Cleansed, Property Type, Bed Type, Amenities, y Cancellation Policy, todas ellas de tipo **object** que deben ser transformadas y/o categorizadas;

- Las variables Host Acceptance Rate y Has Availability pueden ser eliminadas al tener todas sus muestras ausentes; 

- La variable objetivo Price tiene 9 muestras con valores ausentes, que se eliminarán;
       
- La varieble Square Feet debe ser transformada a Square Meter observamos que más de 96% de sus muestras están ausentes, por lo que decidimos eliminar dicha columna;
    
- La varible Zipcode parece tener datos erróneos, reisar y eliminar valores ausente;
    
- Imputar los valores ausentes con la moda en Bed, Bedroom y Bathroom, el 75% de la muestra está dentro del rango, pues parece haber algún autlier;
    
- Sobre las variables Neighbourhood, Neighbourhood Cleansed y Neighbourhood Group Cleansed ver contenido de cada varaibles para decidir si es necesario eliminar alguna de ellas;
    
- Para las variables Weekly Price y Monthly Price analizar con un estudio de correlación y decidir si se pueden eliminar;
    
- Muchas de las variabbles (Accommodates, Square Feet, Weekly Price, Monthly Price, Security Deposit, Number of Reviews, etc) presentan valores dispares, por lo que sería necesario explorar los datos para ecnontrar posibles outliers;
    
- Todas las variables Review excepto Number of Reviews, presentan valores ausentes, la imputación según los datos, puede ser a través de las media o valor meas frecuente. Pero sería necesario valorar la opción de eliminar alguna de dichas variables con un estudio de correlaciones; </span>

<span style="color:#5D8BF4"> **Gráfico de correlaciones** valorar si podemos eliminar alguna variables más en este punto.</span>

In [ ]:
import seaborn as sns

# Compute the correlation matrix
corr = np.abs(df_train.drop(['Price'], axis=1).corr())

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(12, 10))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask,vmin = 0.0, vmax=1.0, center=0.5,
            linewidths=.1, cmap="YlGnBu", cbar_kws={"shrink": .8})

plt.show()

In [ ]:
# eliminamos las columnas con valores ausentes y alta correlación entre variables predictoras
# evitar la colinealidad
df_train = df_train.drop(['Host Acceptance Rate', 'Host Response Time', 'Has Availability', 'Square Feet', 
                          'Neighbourhood', 'Neighbourhood Cleansed', 'Weekly Price', 'Monthly Price', 
                          'Has Availability', 'Availability 30', 'Availability 60', 'Availability 365',
                          'Review Scores Accuracy', 'Review Scores Checkin','Review Scores Communication', 
                          'Reviews per Month', 'Review Scores Value'], axis = 1)

# eliminamos 9 muestras con Price NaN
df_train = df_train.dropna(subset=['Price'])

print(f'Porcentaje de muestras eliminadas: {round((10587-df_train.shape[0])/10587*100,4)} %')
print(f'Tamaño df_train: {df_train.shape}')

<span style="color:#5D8BF4"> Vamos a explorar las variables Property/Room/Bed Type y Cancellation Policy para ver su categorización.<span>

In [ ]:
columns = ['Room Type', 'Bed Type', 'Cancellation Policy', 'Property Type']

for column in columns:
    print (f'{column} --> {df_train[column].unique()}')
    print (f'{column} --> {len(df_train[column].unique())}')
    print (df_train[column].value_counts())

<span style="color:#5D8BF4"> Tomamos las siguientes decisiones:<span>

- **Room Type** pasará a ser variable numérica con 3 valores;
- **Bed Type** pasará a ser variable numérica con 3 valores, dado que las 2 primeros grupos explican el 99% de las muestras, las tres últimas pasaran a formar un sólo grupo;
- **Cancellation Policy** pasará a ser variable numérica con 4 valores, super_strict_60 y super_strict_30 se van a agrupar en un único grupo, que representará al nuevo grupo super-strict;
- **Property Type** pasará a ser variable numérica con 7 valores, dado que los 6 primeros grupos explican más del 98% de las muestras, es decir, el resto de grupos pasaran a formar un solo grupo el de new_other;
    
<span style="color:#5D8BF4"> Aplicamos transformaciones:<span>
    
<span style="color:#5D8BF4"> **Codificación de variables**. Decidimos usar *MeanEncoder* para codificar la variable Neighbourhood Group Cleansed y Property Type. *LeableEncoder* para Room, Bed and Cancellation Policy. </span> 

In [ ]:
# Unificar grupos 
df_train['Bed Type'] = df_train['Bed Type'].str.replace('Futon', 'Other')
df_train['Bed Type'] = df_train['Bed Type'].str.replace('Couch', 'Other')
df_train['Bed Type'] = df_train['Bed Type'].str.replace('Airbed', 'Other')

df_train['Cancellation Policy'] = df_train['Cancellation Policy'].str.replace('super_strict_60', 'super_strict')
df_train['Cancellation Policy'] = df_train['Cancellation Policy'].str.replace('super_strict_30', 'super_strict')

p_types = ['Chalet', 'Guesthouse', 'Dorm', 'Boutique hotel', 'Hostel', 'Serviced apartment',
          'Townhouse', 'Guest suite', 'Earth House', 'Tent', 'Timeshare', 'Villa', 'Casa particular',
          'Camper/RV', 'Bungalow', 'Boat']

for p_type in p_types:
    df_train['Property Type'] = df_train['Property Type'].str.replace(p_type, 'new_Other')

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Codificación de la variables Neighbourhood Cleansed
def calc_smooth_mean(df, by, on, m):
    # Compute the global mean
    mean = df_train[on].mean()

    # Compute the number of values and the mean of each group
    agg = df.groupby(by)[on].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + m * mean) / (counts + m)

    # Replace each value by the according smoothed mean
    return df[by].map(smooth)

df_train['Neighbourhood Group Cleansed'] = calc_smooth_mean(df_train, by='Neighbourhood Group Cleansed', on='Price', m=df_train.shape[0])
df_train['Property Type'] = calc_smooth_mean(df_train, by='Property Type', on='Price', m=df_train.shape[0])


# Codificación de variables con LabelEncoder

#le_prop_type = LabelEncoder()
#le_prop_type.fit(df_train['Property Type'])
#df_train['Property Type'] = le_prop_type.transform(df_train['Property Type'])

le_hrr = LabelEncoder()
le_hrr.fit(df_train['Host Response Rate'])
df_train['Host Response Rate'] = le_hrr.transform(df_train['Host Response Rate'])

le_room_type = LabelEncoder()
le_room_type.fit(df_train['Room Type'])
df_train['Room Type'] = le_room_type.transform(df_train['Room Type'])

le_bed_type = LabelEncoder()
le_bed_type.fit(df_train['Bed Type'])
df_train['Bed Type'] = le_bed_type.transform(df_train['Bed Type'])

le_cancell_p = LabelEncoder()
le_cancell_p.fit(df_train['Cancellation Policy'])
df_train['Cancellation Policy'] = le_cancell_p.transform(df_train['Cancellation Policy'])

<span style="color:#5D8BF4"> **Imputación de valores ausentes** </span>

- Tenemos un neumero bajo de variables a imputar por lo que decidimos tomar la decisión de imputar Bathrooms, Bedrooms y Beds con el valor meas frecuente, aun siendo valor numéricos, no categóricos, su valores puede tender a comportarse como tal. Usaremos mismo prámetros para las variebles Review;
- Las varibles Security Deposit y Cleaning Fee tienen un alto porcetaje de valores perdidos, pero consideramos que ambas variables pueden ser variables predictoras, por ello no las eliminamos. Imputamos los valores perdidos con 0, asuminedo que estas muestras no aplican dichas tarifas;

In [ ]:
from sklearn.impute import SimpleImputer

imputer_mf = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer_mf = imputer_mf.fit(df_train[['Bathrooms', 'Bedrooms', 'Beds', 'Review Scores Rating','Review Scores Cleanliness', 'Review Scores Location']])
df_train = df_train.copy()
df_train[['Bathrooms', 'Bedrooms', 'Beds', 'Review Scores Rating','Review Scores Cleanliness', 'Review Scores Location']] = imputer_mf.transform(df_train[['Bathrooms', 'Bedrooms', 'Beds', 'Review Scores Rating','Review Scores Cleanliness', 'Review Scores Location']])

df_train['Security Deposit'] = df_train['Security Deposit'].fillna(0)
df_train['Cleaning Fee'] = df_train['Cleaning Fee'].fillna(0)

#imputer_mn = SimpleImputer(missing_values=np.nan, strategy='median')
#imputer_mn = imputer_mn.fit(df_p[['Security Deposit', 'Cleaning Fee']])
#df_p = df_p.copy()
#df_p[['Security Deposit', 'Cleaning Fee']] = imputer_mn.transform(df_p[['Security Deposit', 'Cleaning Fee']])


In [ ]:
df_train.isnull().any()

In [ ]:
df_train.dtypes

<span style="color:#5D8BF4"> **Distribuciones, outliers y correlaciones (incluyendo Price)**. Veamos previamente cómo son las variables relaciones y posibles filtrados de los datos. </span> 

In [ ]:
import seaborn as sns

# Compute the correlation matrix
corr = np.abs(df_train.corr())

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(12, 10))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask,vmin = 0.0, vmax=1.0, center=0.5,
            linewidths=.1, cmap="YlGnBu", cbar_kws={"shrink": .8})

plt.show()

<span style="color:#5D8BF4"> Del greafico de correlaciones podemos extraer las siguientes observaiones: </span>

- Las variables Beds y Accommodates parecen que tener una alta correlación (porsible colinealidad), por ahora decidimos no eliminar ninguna de ellas, dado que ambas tienen una buena correlación con la variables objetivo Price, la elminación de una u otra variable se realizará en base a un estudio más exaustiva;
- Dicidimos eliminar las variables Minimun Nights, Maximum Nights, Avialability 90, Number of Reviews al presentar éstas una muy baja correlacieon con la varaible objetivo;
- Ídem para la variable Bed Type;

<span style="color:#5D8BF4"> Eliminamos las variables </span>

In [ ]:
df_train = df_train.drop(['Host Response Rate', 'Minimum Nights', 'Maximum Nights',
                          'Availability 90', 'Number of Reviews', 'Bed Type'], axis = 1)

In [ ]:
df_train.shape

In [ ]:
df_train.describe().T

In [ ]:
columns = df_train.columns
plt.figure(figsize=(30, 30))

for n, col in enumerate(columns):
    plt.subplot(6,4,n+1)
    df_train[col].plot.hist(grid = True)
    plt.axis()
    plt.xlabel(col)

In [ ]:
columns = ['Accommodates', 'Bathrooms', 'Bedrooms', 'Beds', 'Security Deposit', 
           'Cleaning Fee', 'Price', 'Extra People']

plt.figure(figsize=(20, 20))

for n, col in enumerate(columns):
    plt.subplot(3,3,n+1)
    df_train.boxplot(col)

In [ ]:
df_train.plot(kind = 'scatter',x='Accommodates',y = 'Price')
plt.xlabel('# Accommodates')
plt.ylabel('Price (€)')
plt.show()

df_train.plot(kind = 'scatter',x='Bathrooms',y = 'Price')
plt.xlabel('# Bathrooms')
plt.ylabel('Price (€)')
plt.show()

df_train.plot(kind = 'scatter',x='Bedrooms',y = 'Price')
plt.xlabel('# Bedrooms')
plt.ylabel('Price (€)')
plt.show()

df_train.plot(kind = 'scatter',x='Beds',y = 'Price')
plt.xlabel('# Beds')
plt.ylabel('Price (€)')
plt.show()

df_train.plot(kind = 'scatter',x='Security Deposit',y = 'Price')
plt.xlabel('# Security Deposit')
plt.ylabel('Price (€)')
plt.show()

df_train.plot(kind = 'scatter',x='Extra People',y = 'Price')
plt.xlabel('# Extra People')
plt.ylabel('Price (€)')
plt.show()

<span style="color:#5D8BF4"> De los gráficos extraemos las siguientes observaiones: </span>

- Debemos filtrar las variables Accommodates, Bathrooms, Bedrooms, Beds, Security Deposit, Cleaning Fee y Price;
- La variables objetivo Price así como a las variables escoradas hacía los extremos, sería necesario aplicar una transformacieon logarítmica para normalizar sus distribuciones. Se aplicará el logaritmo a las variables cuyo rango está dentro del rango logarítmico, esto es, valores postivisos distintos de 0;

<span style="color:#5D8BF4"> Eliminación de outliers y aplicamos transformaciones</span>

In [ ]:
#df_train['Accommodates'].value_counts()

In [ ]:
df_train = df_train[df_train['Accommodates'] <= 12]
df_train = df_train[df_train['Bathrooms'] <= 6]
df_train = df_train[df_train['Bedrooms'] <= 6]
df_train = df_train[df_train['Beds'] <= 10]
df_train = df_train[df_train['Extra People'] <= 50]
df_train = df_train[df_train['Security Deposit'] <= 600]
df_train = df_train[df_train['Price'] <= 600]

print(f'Porcentaje de muestras eliminadas: {round((10587-df_train.shape[0])/10587*100,4)} %')
print(f'Tamaño df_train: {df_train.shape}')

In [ ]:
#transformación logarítmica de los datos
columns = ['Accommodates', 'Price']

for c in columns:
    df_train[c] = df_train[c].apply(lambda x: np.log10(x))

In [ ]:
df_train.shape

In [ ]:
columns = ['Accommodates', 'Price']
plt.figure(figsize=(30, 30))

for n, col in enumerate(columns):
    plt.subplot(6,4,n+1)
    df_train[col].plot.hist(grid = True)
    plt.axis()
    plt.xlabel(col)

<span style="color:#5D8BF4">Movemos variable objetivo en primera posición.</span>

In [ ]:
first_column = df_train.pop('Price')
df_train.insert(0, "Price", first_column)

df_train.head()

<span style="color:#5D8BF4">**Filtrado y selección de varaibles**</span>

<span style="color:#5D8BF4">Vamos a pasar a ver algunos métodos y comprobar la posibilidad de reducir el número de variables predictoras, para mejorar la intepretabilidad del modelo a desarrollar. Es también probable que con un número alto de variables predictoras es más fácil que el; modelo incurra en overfitting.</span>

In [ ]:
from sklearn.feature_selection import f_regression, mutual_info_regression

# convertimos el DataFrame al formato necesario para scikit-learn
data = df_train.values 

y = data[:,0:1]     # nos quedamos con la 1ª columna, price
X = data[:,1:]      # nos quedamos con el resto

feature_names = df_train.columns[1:]

# do calculations
f_test, _ = f_regression(X, y)
f_test /= np.max(f_test)

mi = mutual_info_regression(X, y)
mi /= np.max(mi)

# do some plotting
plt.figure(figsize=(20, 5))

plt.subplot(1,2,1)
plt.bar(range(X.shape[1]),f_test,  align="center")
plt.xticks(range(X.shape[1]),feature_names, rotation = 90)
plt.xlabel('features')
plt.ylabel('Ranking')
plt.title('$F Test$ score')

plt.subplot(1,2,2)
plt.bar(range(X.shape[1]),mi, align="center")
plt.xticks(range(X.shape[1]),feature_names, rotation = 90)
plt.xlabel('features')
plt.ylabel('Ranking')
plt.title('Mutual information score')

plt.show()